In [117]:
import csv
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [127]:
def read_reports_from_csv():
    df = pd.read_csv("All_Scraped.csv")
    report_list = df["Report"].tolist()
    return report_list

previous_reports = read_reports_from_csv()

In [119]:
options = Options()
# options.add_argument('--headless=new')
# options.add_argument('--ignore-certificate-errors')
driver = webdriver.Chrome(options=options)
driver.maximize_window()

url = 'https://giveaway.com/sponsor'

driver.get(url)
element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.inner')))

In [120]:
def get_data(projects, new_scraped_writer, all_scraped_writer):
    for project in projects:
        report_link = project.get_attribute('href')

        if report_link not in previous_reports:
            data = {}

            data['Name'] = project.find_element(By.CSS_SELECTOR, 'h2.title').text
            data['Report'] = report_link
            
            links = project.find_elements(By.XPATH, './/div[2]/a')
            for link in links:
                style = link.get_attribute('style')
                
                if ('color: rgb(26, 26, 26);' == style):
                    data['Website'] = link.get_attribute('href')

                elif ('color: rgb(28, 187, 239);' == style):
                    data['Telegram'] = link.get_attribute('href')

                elif ('color: rgb(28, 186, 236);' == style):
                    data['Twitter'] = link.get_attribute('href')

                elif ('color: rgb(88, 101, 242);' == style):
                    data['Discord'] = link.get_attribute('href')

            if 'Website' not in data:
                data['Website'] = ''
            elif 'Telegram' not in data:
                data['Telegram'] = ''
            elif 'Twitter' not in data:
                data['Twitter'] = ''
            elif 'Discord' not in data:
                data['Discord'] = ''

            new_scraped_writer.writerow(data)
            all_scraped_writer.writerow(data)
        else:
            continue

In [122]:
header = ['Name','Website','Telegram','Twitter','Discord','Report']
with open("All_Scraped.csv", mode='a', newline='', encoding='utf-8') as all_scraped_file, open("New_Scraped.csv", mode='a', newline='', encoding='utf-8') as new_scraped_file:
    new_scraped_writer = csv.DictWriter(new_scraped_file, fieldnames=header)
    all_scraped_writer = csv.DictWriter(all_scraped_file, fieldnames=header)

    next_button = driver.find_element(By.CSS_SELECTOR, 'button.pagination__btn.next')
    
    while 'disabled' not in next_button.get_attribute('class'):
        projects = driver.find_elements(By.CSS_SELECTOR, 'a.sc-64ff9a54-1.fhJBXs.card')
        get_data(projects, new_scraped_writer, all_scraped_writer)

        next_button.click()
        sleep(5)

    projects = driver.find_elements(By.CSS_SELECTOR, 'a.sc-64ff9a54-1.fhJBXs.card')
    get_data(projects, new_scraped_writer, all_scraped_writer)